# Finance News
from yahoo finance: https://finance.yahoo.com/topic/latest-news/


based on these works
- https://medium.com/@georgemichaeldagogomaynard/web-scraping-dynamic-sites-with-both-selenium-and-beautiful-soup-dim-c679743018de

In [1]:
# Dependencies
# Selenium modules for controlling Chrome browser
from selenium.webdriver import Chrome  # For initializing and controlling the Chrome browser
from selenium import webdriver  # Provides access to the webdriver, allowing interaction with web browsers
from selenium.webdriver.chrome.options import Options  # For configuring Chrome browser options (e.g., headless mode)
from selenium.webdriver.chrome.service import Service  # For managing the ChromeDriver service (e.g., starting, stopping)

# Selenium modules for interacting with web elements
from selenium.webdriver.common.by import By  # For locating elements on a webpage (e.g., By.ID, By.XPATH)
from selenium.webdriver.support.ui import Select  # For interacting with <select> HTML elements (dropdowns)
from selenium.webdriver.support.ui import WebDriverWait  # For implementing explicit waits until a condition is met
from selenium.webdriver.support import expected_conditions as EC  # For defining conditions to wait for (e.g., element visibility)

# Other useful libraries
from fake_useragent import UserAgent  # For generating random user agents to mimic different browsers
import time  # For adding delays (e.g., time.sleep) during the script execution
import requests  # For making HTTP requests to interact with websites directly without using a browser
from bs4 import BeautifulSoup  # For parsing and extracting data from HTML content
import pandas as pd  # For data manipulation, analysis, and creating DataFrames

In [ ]:
# Testing the scraper (replace print with storing data in a structured format later)
import re

news_uri = "https://finance.yahoo.com/topic/latest-news/"

# Add headers to mimic a browser request to avoid 429 Too Many Requests
ua = UserAgent()
headers = {'User-Agent': ua.random}

news_page = requests.get(news_uri, headers=headers)
#print (page.status_code)  # should be 200 if successful
#print (page.content[:100])  

news_soup = BeautifulSoup(news_page.content, 'html.parser')

news_list = news_soup.find('ul', class_= "stream-items yf-9xydx9")
news_listings = news_list.find_all('li', class_= "stream-item story-item yf-9xydx9")

# TODO: catch None conditions, make the driver scroll to load more news, fix duplicates issue
for news in news_listings:
    # Want to scrape: title, link, publisher, tickers 
    print(news.find('h3').text)  # Article's title
    
    # Use regex to match 'publishing' followed by any other class names
    print(news.find('div', class_=re.compile(r'publishing.*')).text.split('•')[0]) # Publisher name
    if news.find('div', class_='taxonomy-links'):  
        tickers = []
        for ticker in news.find('div', class_='taxonomy-links').find_all('span', class_=re.compile(r'ticker-wrapper.*')):
            tickers.append(ticker.text.strip())   
            #print(ticker.text.strip())   # Each ticker name
        print (', '.join(tickers))  # All tickers in one line
    else:
        print("No tickers found")
    print (news.find('a')['href']) # link to the article
    
    #bug is after here?
    #TODO: make this process faster
    news_detail_url = news.find('a')['href']
    news_detail_page = requests.get(news_detail_url, headers=headers)
    news_detail_soup = BeautifulSoup(news_detail_page.content, 'html.parser')
    
    # writers, time published, article content, 
    # TODO: might use ticker-wrapper from <div class= "slide"> instead
    print(news_detail_soup.find('div', class_=re.compile(r'byline-attr-author.*')).text.strip()) # Authors
    print(news_detail_soup.find('time').text) # Time published
    
    # bug is caused before here?
    
    # Body: for each paragraph in the article content
    body_wrapper = news_detail_soup.find('div', class_='bodyItems-wrapper')
    if body_wrapper:
        for para in body_wrapper.find_all('p'):
            # Check if the paragraph is inside 'read-more-wrapper' to avoid duplication
            if para.find_parent('div', class_='read-more-wrapper'): # if parent is read-more-wrapper
                continue
            print(para.text.strip())
            
    # 'Read more' section
    read_more_wrapper = news_detail_soup.find('div', class_='read-more-wrapper')
    if read_more_wrapper:
        for para in read_more_wrapper.find_all('p'): 
            print(para.text.strip())
    
    
    print ("\n")
# ul and li(class = story-item) tags are used for listing news articles
# For the content of each news article: <div class=content> ->...(below)..
# inside div(class = footer) ->  <div(class = publishing)> for publisher name and time (1st & 2nd text)
# inside div(class = footer) -> <div(class = taxonomy-links)> -> <div(class=ticker-wrapper)> for each ticker

Micron to invest $9.6 billion in Japan to build AI memory chip plant, Nikkei reports
Reuters 
MU
https://finance.yahoo.com/news/micron-invest-9-6-billion-075214757.html
Reuters
Sat, November 29, 2025 at 2:52 PM GMT+7
Nov 29 (Reuters) -​ U.S.‌ chipmaker Micron Technology ‌will invest 1.5 trillion yen ($9.6 billion)⁠ ‌to build a next-generation memory ‍manufacturing facility in western Japan to support artificial ​intelligence computing, ‌Nikkei reported on Saturday
Reuters could not immediately verify the ⁠report, ​which cited ​people familiar with the matter.
($1 = ‍156.⁠1500 yen)
(Reporting by Rajveer ⁠Singh Pardesi in Bengaluru;‌ Editing by ‌William Mallard)


‘Does Reeves have a plan for growth?’ Your Budget questions answered
The Telegraph 
No tickers found
https://finance.yahoo.com/news/does-reeves-plan-growth-budget-070000118.html
Rachel Obordo
Sat, November 29, 2025 at 2:00 PM GMT+7
b'
'
Following a £30bn raid in this year’s Budget, Rachel Reeves has refused to rule out the poss

In [23]:
soup.prettify()[:200] 

'<!DOCTYPE html>\n<html class="smartphone neo-green dock-upscale" data-color-scheme="auto" data-color-theme-enabled="true" lang="en-US" theme="auto">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="g'

In [ ]:
# Connect to MongoDB
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv

load_dotenv()
db_username = os.getenv("db_username")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")

if not all([db_username, db_password, db_host]):
    print("Please set db_username, db_password, and db_host in your .env file")
else:
    uri = f"mongodb+srv://{db_username}:{db_password}@{db_host}"
    client = MongoClient(uri, server_api=ServerApi('1'))

    try:
        client.admin.command('ping')
        print("Pinged your deployment. You successfully connected to MongoDB!")
        
        # Create/Select database and collection
        db = client["finance_news_db"] 
        collection = db["news_articles"]
        print(f"Connected to database: {db.name}, collection: {collection.name}")
        
    except Exception as e:
        print(e)

Pinged your deployment. You successfully connected to MongoDB!
Connected to database: finance_news_db, collection: sample_articles


In [ ]:
# Working scraper!
import re

news_uri = "https://finance.yahoo.com/topic/latest-news/"

# Add headers to mimic a browser request to avoid 429 Too Many Requests
ua = UserAgent()
headers = {'User-Agent': ua.random}

news_page = requests.get(news_uri, headers=headers)
#print (page.status_code)  # should be 200 if successful
#print (page.content[:100])  

news_soup = BeautifulSoup(news_page.content, 'html.parser')

news_list = news_soup.find('ul', class_= "stream-items yf-9xydx9")
news_listings = news_list.find_all('li', class_= "stream-item story-item yf-9xydx9")

# TODO: catch None conditions, make the driver scroll to load more news 
for news in news_listings:
    news_data = {}
    
    # Want to scrape: title, link, publisher, tickers 
    news_data['title'] = news.find('h3').text
    
    # Use regex to match 'publishing' followed by any other class names
    news_data['publisher'] = news.find('div', class_=re.compile(r'publishing.*')).text.split('•')[0]
    
    tickers = []
    if news.find('div', class_='taxonomy-links'):  
        for ticker in news.find('div', class_='taxonomy-links').find_all('span', class_=re.compile(r'ticker-wrapper.*')):
            tickers.append(ticker.text.strip())   
    news_data['tickers'] = tickers
    
    news_data['link'] = news.find('a')['href']
    
    #TODO: make this process faster
    news_detail_url = news.find('a')['href']
    news_detail_page = requests.get(news_detail_url, headers=headers)
    news_detail_soup = BeautifulSoup(news_detail_page.content, 'html.parser')
    
    # Want to scrape: writers, time published, article content(body)
    # TODO: might use ticker-wrapper from <div class= "slide"> instead
    
    # Check if author exists
    author_div = news_detail_soup.find('div', class_=re.compile(r'byline-attr-author.*'))
    news_data['authors'] = author_div.text.strip() if author_div else "Unknown"

    # Check if time exists
    time_tag = news_detail_soup.find('time')
    news_data['time_published'] = time_tag.text if time_tag else "Unknown"
    
    
    # Body: for each paragraph in the article content
    content_text = []
    body_wrapper = news_detail_soup.find('div', class_='bodyItems-wrapper')
    seen_texts = set() # Keep track of printed text to avoid duplicates
    
    if body_wrapper:
        for para in body_wrapper.find_all('p'):
            text = para.text.strip()
            
            # Skip if text is empty or already seen
            if not text or text in seen_texts:
                continue
            
            # Stop processing if we hit the "Most Read" or "Recommended" sections
            if "Most Read from" in text or "Recommended Stories" in text:
                break

            # Check if the paragraph is inside 'read-more-wrapper' to avoid duplication
            if para.find_parent('div', class_='read-more-wrapper'): # if parent is read-more-wrapper, skip
                continue
            
            content_text.append(text)
            seen_texts.add(text)
            
    news_data['content'] = "\n".join(content_text)
    
    # Insert into MongoDB
    # Check if already exists to avoid duplicates (using link as unique identifier)
    if collection.count_documents({'link': news_data['link']}) == 0:
        collection.insert_one(news_data)
        print(f"Inserted: {news_data['title']}")
    else:
        print(f"Skipped (already exists): {news_data['title']}")
    
    print ("\n")
# ul and li(class = story-item) tags are used for listing news articles
# For the content of each news article: <div class=content> ->...(below)..
# inside div(class = footer) ->  <div(class = publishing)> for publisher name and time (1st & 2nd text)
# inside div(class = footer) -> <div(class = taxonomy-links)> -> <div(class=ticker-wrapper)> for each ticker

Inserted: Micron to invest $9.6 billion in Japan to build AI memory chip plant, Nikkei reports


Inserted: ‘Does Reeves have a plan for growth?’ Your Budget questions answered


Inserted: VC Kara Nortman bet early on women’s sports, and now she’s creating the market


Inserted: Could the EU’s frozen-assets plan really destabilise European bond markets?


Inserted: Ryanair scraps VIP scheme after customers take too many cheap flights


Inserted: This African nation built its development on diamonds. Now it's crashing down


Inserted: A couple earning six figures from 28 rental units explains how they use a 'buy box' to ensure positive cash flow


Inserted: Michael Burry just started a massive group chat on Substack, and it's as chaotic as you'd expect


Inserted: Shoppers are on pace to break Black Friday online spending records and use AI more than ever as sales hit $8.6 billion


Inserted: 3 Highest-Yielding Dividend Kings To Buy, Hold, and Forget


Inserted: Tesla loses some AI staff